In [38]:
import pandas as pd
import json
import requests
import itertools
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup
from tqdm import tqdm

# Crawling


## Data Collection with API

### Youtube

site: https://console.developers.google.com/ <br>
reference: https://developers.google.com/youtube/v3/docs?hl=ko <br>

```python
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from oauth2client.tools import argparser

import pandas as pd


DEVELOPER_KEY=''
YOUTUBE_API_SERVICE_NAME='youtube'
YOUTUBE_API_VERSION='v3'

youtube = build(
  YOUTUBE_API_SERVICE_NAME,
  YOUTUBE_API_VERSION,
  developerKey=DEVELOPER_KEY
  )
```

In [ ]:
!pip install google-api-python-client
!pip install oauth2client

In [2]:
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from oauth2client.tools import argparser

DEVELOPER_KEY = ''
YOUTUBE_API_SERVICE_NAME = 'youtube'
YOUTUBE_API_VERSION = 'v3'

youtube = build(
  YOUTUBE_API_SERVICE_NAME,
  YOUTUBE_API_VERSION,
  developerKey=DEVELOPER_KEY
  )

#### Videos

채널 내 비디오 조회 기본 코드
```python
response = (
  youtube
  .playlistItems()
  .list(
    playlistId=*channel_id*,
    part='snippet',
    maxResults=50
    )
  .execute()
  )
```


1번의 query에 전체 데이터를 다 받을 수 없음 <br>
따라서 pageToken을 이용하여 다음 페이지의 정보를 받아야 함 <br>
```python
if 'nextPageToken' in response:
  response = (
    youtube
    .playlistItems()
    .list(
      playlistId=*channel_id*,
      pageToken=response.get('nextPageToken'),
      part='snippet',
      maxResults=50
      )
    .execute()
    )
```

#### Comments
비디오 내 댓글 조회 기본 코드
```python
response = (
    youtube
    .commentThreads()
    .list(
      part='snippet,replies', 
      videoId=*video_id*, 
      maxResults=100
      )
    .execute()
    )
```

1번의 query에 전체 데이터를 다 받을 수 없음 <br>
따라서 pageToken을 이용하여 다음 페이지의 정보를 받아야 함 <br>
```python
if 'nextPageToken' in response:
  response = (
    youtube
    .commentThreads()
    .list(
      part='snippet,replies',
      videoId=video_id,
      pageToken=response.get('nextPageToken'),
      maxResults=100
      )
    .execute()
    )
```

In [17]:
response = (
    youtube
    .commentThreads()
    .list(
      part='snippet,replies', 
      videoId='DLZI557Zrgs', 
      maxResults=100
      )
    .execute()
    )

In [65]:
response.get('items')[0].keys()

dict_keys(['kind', 'etag', 'id', 'snippet', 'replies'])

In [50]:
response.get('items')[0].get('snippet').get('topLevelComment').get('snippet')

{'channelId': 'UCGX5sP4ehBkihHwt5bs5wvg',
 'videoId': 'DLZI557Zrgs',
 'textDisplay': '🌶️ 2024 영양 H.O.T 페스티벌 2024. 8. 29 (목) ~ 8. 31 (토) 🌶️<br>피식대학X영양군 콘텐츠가 앞으로 2주간 업로드 됩니다! 많은 기대와 관심 부탁드립니다! 🫶',
 'textOriginal': '🌶️ 2024 영양 H.O.T 페스티벌 2024. 8. 29 (목) ~ 8. 31 (토) 🌶️\n피식대학X영양군 콘텐츠가 앞으로 2주간 업로드 됩니다! 많은 기대와 관심 부탁드립니다! 🫶',
 'authorDisplayName': '@hot_yyg',
 'authorProfileImageUrl': 'https://yt3.ggpht.com/StOaeEub9ecLIRTseLzDkke9eEiJiAWSZKrdCkA-j7XlHYo-bBXPovfnmrRIHAEJgR17cNfq=s48-c-k-c0x00ffffff-no-rj',
 'authorChannelUrl': 'http://www.youtube.com/@hot_yyg',
 'authorChannelId': {'value': 'UCst9o0ksMThIfzMluNvfUXQ'},
 'canRate': True,
 'viewerRating': 'none',
 'likeCount': 4669,
 'publishedAt': '2024-08-20T10:04:17Z',
 'updatedAt': '2024-08-20T10:04:17Z'}

In [51]:
# response.get('items')[0].get('snippet').get('topLevelComment').get('snippet')
[
    {
        'channelId': item.get('snippet').get('topLevelComment').get('snippet').get('channelId'),
        'videoId': item.get('snippet').get('topLevelComment').get('snippet').get('videoId'),
        'textDisplay': item.get('snippet').get('topLevelComment').get('snippet').get('textDisplay'),
        'textOriginal': item.get('snippet').get('topLevelComment').get('snippet').get('textOriginal'),
        'authorDisplayName': item.get('snippet').get('topLevelComment').get('snippet').get('authorDisplayName'),
        'likeCount': item.get('snippet').get('topLevelComment').get('snippet').get('likeCount'),
        'publishedAt': item.get('snippet').get('topLevelComment').get('snippet').get('publishedAt'),
        'updatedAt': item.get('snippet').get('topLevelComment').get('snippet').get('updatedAt'),
    }
    for item
    in response.get('items')
]

[{'channelId': 'UCGX5sP4ehBkihHwt5bs5wvg',
  'videoId': 'DLZI557Zrgs',
  'textDisplay': '🌶️ 2024 영양 H.O.T 페스티벌 2024. 8. 29 (목) ~ 8. 31 (토) 🌶️<br>피식대학X영양군 콘텐츠가 앞으로 2주간 업로드 됩니다! 많은 기대와 관심 부탁드립니다! 🫶',
  'textOriginal': '🌶️ 2024 영양 H.O.T 페스티벌 2024. 8. 29 (목) ~ 8. 31 (토) 🌶️\n피식대학X영양군 콘텐츠가 앞으로 2주간 업로드 됩니다! 많은 기대와 관심 부탁드립니다! 🫶',
  'authorDisplayName': '@hot_yyg',
  'likeCount': 4669,
  'publishedAt': '2024-08-20T10:04:17Z',
  'updatedAt': '2024-08-20T10:04:17Z'},
 {'channelId': 'UCGX5sP4ehBkihHwt5bs5wvg',
  'videoId': 'DLZI557Zrgs',
  'textDisplay': '이야, 기획 잘했다 진짜 <br>나락이랑 극락을 왔다갔다 해버리네',
  'textOriginal': '이야, 기획 잘했다 진짜 \n나락이랑 극락을 왔다갔다 해버리네',
  'authorDisplayName': '@user-lt6bw6nu9d',
  'likeCount': 0,
  'publishedAt': '2024-08-21T01:04:18Z',
  'updatedAt': '2024-08-21T01:04:18Z'},
 {'channelId': 'UCGX5sP4ehBkihHwt5bs5wvg',
  'videoId': 'DLZI557Zrgs',
  'textDisplay': '획기적인<br>아이디어에 웃음이 나오네요 화이팅!!',
  'textOriginal': '획기적인\n아이디어에 웃음이 나오네요 화이팅!!',
  'authorDisplayName': '@user-nr7xz4pe6x',
  

In [5]:
def extract_comments(item):
    return {
        'channelId': item.get('snippet').get('topLevelComment').get('snippet').get('channelId'),
        'videoId': item.get('snippet').get('topLevelComment').get('snippet').get('videoId'),
        'parentID': '', 
        'textDisplay': item.get('snippet').get('topLevelComment').get('snippet').get('textDisplay'),
        'textOriginal': item.get('snippet').get('topLevelComment').get('snippet').get('textOriginal'),
        'authorDisplayName': item.get('snippet').get('topLevelComment').get('snippet').get('authorDisplayName'),
        'likeCount': item.get('snippet').get('topLevelComment').get('snippet').get('likeCount'),
        'publishedAt': item.get('snippet').get('topLevelComment').get('snippet').get('publishedAt'),
        'updatedAt': item.get('snippet').get('topLevelComment').get('snippet').get('updatedAt'),
    }

In [53]:
[extract_comments(item) for item in response.get('items')]

[{'channelId': 'UCGX5sP4ehBkihHwt5bs5wvg',
  'videoId': 'DLZI557Zrgs',
  'textDisplay': '🌶️ 2024 영양 H.O.T 페스티벌 2024. 8. 29 (목) ~ 8. 31 (토) 🌶️<br>피식대학X영양군 콘텐츠가 앞으로 2주간 업로드 됩니다! 많은 기대와 관심 부탁드립니다! 🫶',
  'textOriginal': '🌶️ 2024 영양 H.O.T 페스티벌 2024. 8. 29 (목) ~ 8. 31 (토) 🌶️\n피식대학X영양군 콘텐츠가 앞으로 2주간 업로드 됩니다! 많은 기대와 관심 부탁드립니다! 🫶',
  'authorDisplayName': '@hot_yyg',
  'likeCount': 4669,
  'publishedAt': '2024-08-20T10:04:17Z',
  'updatedAt': '2024-08-20T10:04:17Z'},
 {'channelId': 'UCGX5sP4ehBkihHwt5bs5wvg',
  'videoId': 'DLZI557Zrgs',
  'textDisplay': '이야, 기획 잘했다 진짜 <br>나락이랑 극락을 왔다갔다 해버리네',
  'textOriginal': '이야, 기획 잘했다 진짜 \n나락이랑 극락을 왔다갔다 해버리네',
  'authorDisplayName': '@user-lt6bw6nu9d',
  'likeCount': 0,
  'publishedAt': '2024-08-21T01:04:18Z',
  'updatedAt': '2024-08-21T01:04:18Z'},
 {'channelId': 'UCGX5sP4ehBkihHwt5bs5wvg',
  'videoId': 'DLZI557Zrgs',
  'textDisplay': '획기적인<br>아이디어에 웃음이 나오네요 화이팅!!',
  'textOriginal': '획기적인\n아이디어에 웃음이 나오네요 화이팅!!',
  'authorDisplayName': '@user-nr7xz4pe6x',
  

In [57]:
temp = []
for item in response.get('items'):
    temp.append({
        'channelId': item.get('snippet').get('topLevelComment').get('snippet').get('channelId'),
        'videoId': item.get('snippet').get('topLevelComment').get('snippet').get('videoId'),
        'textDisplay': item.get('snippet').get('topLevelComment').get('snippet').get('textDisplay'),
        'textOriginal': item.get('snippet').get('topLevelComment').get('snippet').get('textOriginal'),
        'authorDisplayName': item.get('snippet').get('topLevelComment').get('snippet').get('authorDisplayName'),
        'likeCount': item.get('snippet').get('topLevelComment').get('snippet').get('likeCount'),
        'publishedAt': item.get('snippet').get('topLevelComment').get('snippet').get('publishedAt'),
        'updatedAt': item.get('snippet').get('topLevelComment').get('snippet').get('updatedAt'),
    })

In [58]:
pd.DataFrame(temp)

,channelId,videoId,textDisplay,textOriginal,authorDisplayName,likeCount,publishedAt,updatedAt
0,UCGX5sP4ehBkihHwt5bs5wvg,DLZI557Zrgs,🌶️ 2024 영양 H.O.T 페스티벌 2024. 8. 29 (목) ~ 8. 31 ...,🌶️ 2024 영양 H.O.T 페스티벌 2024. 8. 29 (목) ~ 8. 31 ...,@hot_yyg,4669,2024-08-20T10:04:17Z,2024-08-20T10:04:17Z
1,UCGX5sP4ehBkihHwt5bs5wvg,DLZI557Zrgs,"이야, 기획 잘했다 진짜 <br>나락이랑 극락을 왔다갔다 해버리네","이야, 기획 잘했다 진짜 \n나락이랑 극락을 왔다갔다 해버리네",@user-lt6bw6nu9d,0,2024-08-21T01:04:18Z,2024-08-21T01:04:18Z
2,UCGX5sP4ehBkihHwt5bs5wvg,DLZI557Zrgs,획기적인<br>아이디어에 웃음이 나오네요 화이팅!!,획기적인\n아이디어에 웃음이 나오네요 화이팅!!,@user-nr7xz4pe6x,0,2024-08-21T01:04:17Z,2024-08-21T01:04:17Z
3,UCGX5sP4ehBkihHwt5bs5wvg,DLZI557Zrgs,"<a href=""https://www.youtube.com/watch?v=DLZI5...",1:30 혼자 빠르다가 동화돼서 느려진거 왤케 웃김,@B_byong,0,2024-08-21T01:03:27Z,2024-08-21T01:03:27Z
4,UCGX5sP4ehBkihHwt5bs5wvg,DLZI557Zrgs,ㅈㄴ웃기네,ㅈㄴ웃기네,@dsgsfizkwdnzh,0,2024-08-21T01:02:42Z,2024-08-21T01:02:42Z
...,...,...,...,...,...,...,...,...
95,UCGX5sP4ehBkihHwt5bs5wvg,DLZI557Zrgs,ㅅㅂ ㅈㄴ 역겹네 ㅋㅋㅋㅋㅋ,ㅅㅂ ㅈㄴ 역겹네 ㅋㅋㅋㅋㅋ,@shpp9161,15,2024-08-20T23:21:05Z,2024-08-20T23:21:05Z
96,UCGX5sP4ehBkihHwt5bs5wvg,DLZI557Zrgs,처음부터 홍보해줬으면 얼마나 좋아 욕도 안먹고 욕먹고 나서 조회수 나락가고 구독자 ...,처음부터 홍보해줬으면 얼마나 좋아 욕도 안먹고 욕먹고 나서 조회수 나락가고 구독자 ...,@user-yw7fh7rx4l,11,2024-08-20T23:20:07Z,2024-08-20T23:20:07Z
97,UCGX5sP4ehBkihHwt5bs5wvg,DLZI557Zrgs,비난 받으면 그저 불복하고 그건 니들이 잘못한거고~ 하며 사상의 불손을 이야기하는 ...,비난 받으면 그저 불복하고 그건 니들이 잘못한거고~ 하며 사상의 불손을 이야기하는 ...,@yongheeshin7938,1,2024-08-20T23:19:30Z,2024-08-20T23:19:30Z
98,UCGX5sP4ehBkihHwt5bs5wvg,DLZI557Zrgs,"정말 피식대학을 사랑하는 오랜 구독자이지만, 솔직히 그동안은 부끄러워서 영상을 한개...","정말 피식대학을 사랑하는 오랜 구독자이지만, 솔직히 그동안은 부끄러워서 영상을 한개...",@user-gd9ln4cq1x,0,2024-08-20T23:19:16Z,2024-08-20T23:21:09Z


In [61]:
##### 전체 댓글

response = (
    youtube
    .commentThreads()
    .list(
      part='snippet,replies', 
      videoId='DLZI557Zrgs', 
      maxResults=100
      )
    .execute()
    )

comments = [extract_comments(item) for item in response.get('items')]

# 2page ~ 
while 'nextPageToken' in response:
    response = (
        youtube
        .commentThreads()
        .list(
          part='snippet,replies', 
          videoId='DLZI557Zrgs', 
          maxResults=100,
          pageToken=response.get('nextPageToken'),
          )
        .execute()
        )
    
    comments.extend([extract_comments(item) for item in response.get('items')])

In [85]:
##### 대댓글

98

In [92]:
response = (
    youtube
    .commentThreads()
    .list(
      part='snippet,replies', 
      videoId='DLZI557Zrgs', 
      maxResults=100
      )
    .execute()
    )

len(response.get('items')[0].get('replies').get('comments'))

5

In [98]:
response.get('items')[0].get('replies').get('comments')[0].get('snippet')

{'channelId': 'UCGX5sP4ehBkihHwt5bs5wvg',
 'videoId': 'DLZI557Zrgs',
 'textDisplay': '원상복구 이상의 노력이다 응원해요',
 'textOriginal': '원상복구 이상의 노력이다 응원해요',
 'parentId': 'UgylQQlbn0xpRjkyz_14AaABAg',
 'authorDisplayName': '@utopia-de5my',
 'authorProfileImageUrl': 'https://yt3.ggpht.com/ytc/AIdro_nR1gPUogELqARyG-TwuXgcso4RuA9Gf0FP6DJK2AAllGA2meJnS1Tb5kQghVpWZiW-5w=s48-c-k-c0x00ffffff-no-rj',
 'authorChannelUrl': 'http://www.youtube.com/@utopia-de5my',
 'authorChannelId': {'value': 'UC1bC7oZD_WcvqNlGFko5OEA'},
 'canRate': True,
 'viewerRating': 'none',
 'likeCount': 270,
 'publishedAt': '2024-08-20T10:05:06Z',
 'updatedAt': '2024-08-20T10:05:06Z'}

In [6]:
def _extract_reply(reply):
    if reply:
        return {
        'channelId': reply.get('snippet').get('channelId'),
        'videoId': reply.get('snippet').get('videoId'),
        'parentId': reply.get('snippet').get('parentId'),
        'textDisplay': reply.get('snippet').get('textDisplay'),
        'textOriginal': reply.get('snippet').get('textOriginal'),
        'authorDisplayName': reply.get('snippet').get('authorDisplayName'),
        'likeCount': reply.get('snippet').get('likeCount'),
        'publishedAt': reply.get('snippet').get('publishedAt'),
        'updatedAt': reply.get('snippet').get('updatedAt'),
        }

In [136]:
response.get('items')[0].get('replies').get('comments')[0].get('snippet')

{'channelId': 'UCGX5sP4ehBkihHwt5bs5wvg',
 'videoId': 'DLZI557Zrgs',
 'textDisplay': '원상복구 이상의 노력이다 응원해요',
 'textOriginal': '원상복구 이상의 노력이다 응원해요',
 'parentId': 'UgylQQlbn0xpRjkyz_14AaABAg',
 'authorDisplayName': '@utopia-de5my',
 'authorProfileImageUrl': 'https://yt3.ggpht.com/ytc/AIdro_nR1gPUogELqARyG-TwuXgcso4RuA9Gf0FP6DJK2AAllGA2meJnS1Tb5kQghVpWZiW-5w=s48-c-k-c0x00ffffff-no-rj',
 'authorChannelUrl': 'http://www.youtube.com/@utopia-de5my',
 'authorChannelId': {'value': 'UC1bC7oZD_WcvqNlGFko5OEA'},
 'canRate': True,
 'viewerRating': 'none',
 'likeCount': 270,
 'publishedAt': '2024-08-20T10:05:06Z',
 'updatedAt': '2024-08-20T10:05:06Z'}

In [7]:
def extract_replies(items):
    if items:
        return [_extract_reply(item) for item in items]
    else:
        return []

In [143]:
replies = [extract_replies(item.get('replies').get('comments')) for item in response.get('items') if item.get('replies')]

In [150]:
replies

[[{'channelId': 'UCGX5sP4ehBkihHwt5bs5wvg',
   'videoId': 'DLZI557Zrgs',
   'parentId': 'UgylQQlbn0xpRjkyz_14AaABAg',
   'textDisplay': '원상복구 이상의 노력이다 응원해요',
   'textOriginal': '원상복구 이상의 노력이다 응원해요',
   'authorDisplayName': '@utopia-de5my',
   'likeCount': 270,
   'publishedAt': '2024-08-20T10:05:06Z',
   'updatedAt': '2024-08-20T10:05:06Z'},
  {'channelId': 'UCGX5sP4ehBkihHwt5bs5wvg',
   'videoId': 'DLZI557Zrgs',
   'parentId': 'UgylQQlbn0xpRjkyz_14AaABAg',
   'textDisplay': '멋있다.',
   'textOriginal': '멋있다.',
   'authorDisplayName': '@user-nu9qb1yu5r',
   'likeCount': 58,
   'publishedAt': '2024-08-20T10:07:40Z',
   'updatedAt': '2024-08-20T10:07:40Z'},
  {'channelId': 'UCGX5sP4ehBkihHwt5bs5wvg',
   'videoId': 'DLZI557Zrgs',
   'parentId': 'UgylQQlbn0xpRjkyz_14AaABAg',
   'textDisplay': '피식대학 좋아요~',
   'textOriginal': '피식대학 좋아요~',
   'authorDisplayName': '@aaaa-o6g5f',
   'likeCount': 47,
   'publishedAt': '2024-08-20T10:08:04Z',
   'updatedAt': '2024-08-20T10:08:04Z'},
  {'channelId':

In [151]:
replies = list(itertools.chain(*replies))

In [153]:
pd.DataFrame(replies)

,channelId,videoId,parentId,textDisplay,textOriginal,authorDisplayName,likeCount,publishedAt,updatedAt
0,UCGX5sP4ehBkihHwt5bs5wvg,DLZI557Zrgs,UgylQQlbn0xpRjkyz_14AaABAg,원상복구 이상의 노력이다 응원해요,원상복구 이상의 노력이다 응원해요,@utopia-de5my,270,2024-08-20T10:05:06Z,2024-08-20T10:05:06Z
1,UCGX5sP4ehBkihHwt5bs5wvg,DLZI557Zrgs,UgylQQlbn0xpRjkyz_14AaABAg,멋있다.,멋있다.,@user-nu9qb1yu5r,58,2024-08-20T10:07:40Z,2024-08-20T10:07:40Z
2,UCGX5sP4ehBkihHwt5bs5wvg,DLZI557Zrgs,UgylQQlbn0xpRjkyz_14AaABAg,피식대학 좋아요~,피식대학 좋아요~,@aaaa-o6g5f,47,2024-08-20T10:08:04Z,2024-08-20T10:08:04Z
3,UCGX5sP4ehBkihHwt5bs5wvg,DLZI557Zrgs,UgylQQlbn0xpRjkyz_14AaABAg,🎉🎉🎉영양 핫 페스티발 ❤,🎉🎉🎉영양 핫 페스티발 ❤,@user-hy5ch6pn2y,22,2024-08-20T10:08:55Z,2024-08-20T10:08:55Z
4,UCGX5sP4ehBkihHwt5bs5wvg,DLZI557Zrgs,UgylQQlbn0xpRjkyz_14AaABAg,ㅅㅂ존나 멋있네,ㅅㅂ존나 멋있네,@user-tc6om2gf5x,34,2024-08-20T10:18:07Z,2024-08-20T10:18:07Z


In [166]:
##### 전체 댓글 + 대댓글

response = (
    youtube
    .commentThreads()
    .list(
      part='snippet,replies', 
      videoId='DLZI557Zrgs', 
      maxResults=100
      )
    .execute()
    )

comments = [extract_comments(item) for item in response.get('items')]
replies = [extract_replies(item.get('replies').get('comments')) for item in response.get('items') if item.get('replies')]
replies = list(itertools.chain(*replies))

# 2page ~ 
while 'nextPageToken' in response:
    response = (
        youtube
        .commentThreads()
        .list(
          part='snippet,replies', 
          videoId='DLZI557Zrgs', 
          maxResults=100,
          pageToken=response.get('nextPageToken'),
          )
        .execute()
        )
    
    comments.extend([extract_comments(item) for item in response.get('items')])
    _replies = [extract_replies(item.get('replies').get('comments')) for item in response.get('items') if item.get('replies')]
    _replies = list(itertools.chain(*_replies))
    replies.extend(_replies)

KeyboardInterrupt: 

In [167]:
response = (
    youtube
    .commentThreads()
    .list(
      part='snippet,replies', 
      videoId='JdRcM4fLwgE', 
      maxResults=100
      )
    .execute()
    )

comments = [extract_comments(item) for item in response.get('items')]
replies = [extract_replies(item.get('replies').get('comments')) for item in response.get('items') if item.get('replies')]
replies = list(itertools.chain(*replies))

# 2page ~ 
while 'nextPageToken' in response:
    response = (
        youtube
        .commentThreads()
        .list(
          part='snippet,replies', 
          videoId='JdRcM4fLwgE', 
          maxResults=100,
          pageToken=response.get('nextPageToken'),
          )
        .execute()
        )
    
    comments.extend([extract_comments(item) for item in response.get('items')])
    _replies = [extract_replies(item.get('replies').get('comments')) for item in response.get('items') if item.get('replies')]
    _replies = list(itertools.chain(*_replies))
    replies.extend(_replies)

In [170]:
len(comments)

1946

In [171]:
len(replies)

394

In [11]:
def crawl_youtube_comments(video_id):
	response = (
		youtube
		.commentThreads()
		.list(
		part='snippet,replies',
		videoId=video_id, 
		maxResults=100
		)
		.execute()
		)

	comments = [extract_comments(item) for item in response.get('items')]
	replies = [extract_replies(item.get('replies').get('comments')) for item in response.get('items') if item.get('replies')]
	replies = list(itertools.chain(*replies))

	# 2page ~ 
	while 'nextPageToken' in response:
		response = (
			youtube
			.commentThreads()
			.list(
				part='snippet,replies', 
				videoId=video_id, 
				maxResults=100,
				pageToken=response.get('nextPageToken'),
			)
			.execute()
			)
		
		comments.extend([extract_comments(item) for item in response.get('items')])
		_replies = [extract_replies(item.get('replies').get('comments')) for item in response.get('items') if item.get('replies')]
		_replies = list(itertools.chain(*_replies))
		replies.extend(_replies)
	
	return comments, replies

In [12]:
crawl_youtube_comments('Tgeia3MrEHc')

([{'channelId': 'UCWiS4vemV0wcxuxYPx9Z62g',
   'videoId': 'Tgeia3MrEHc',
   'parentID': '',
   'textDisplay': '사과는 담당과장이 해야 맞고... 아님 지금 사과영상에 시장님이라도 같이 나왔어야 멋있지.... 지금 이거 너무 노간지인뎅.',
   'textOriginal': '사과는 담당과장이 해야 맞고... 아님 지금 사과영상에 시장님이라도 같이 나왔어야 멋있지.... 지금 이거 너무 노간지인뎅.',
   'authorDisplayName': '@LilacASMR-ASMR',
   'likeCount': 0,
   'publishedAt': '2024-08-21T05:10:13Z',
   'updatedAt': '2024-08-21T05:10:13Z'},
  {'channelId': 'UCWiS4vemV0wcxuxYPx9Z62g',
   'videoId': 'Tgeia3MrEHc',
   'parentID': '',
   'textDisplay': '실망? 아니. 어이가 없다.<br>지껄이는 꼴이..',
   'textOriginal': '실망? 아니. 어이가 없다.\n지껄이는 꼴이..',
   'authorDisplayName': '@user-bs8dn6ni1q',
   'likeCount': 0,
   'publishedAt': '2024-08-21T05:10:03Z',
   'updatedAt': '2024-08-21T05:10:03Z'},
  {'channelId': 'UCWiS4vemV0wcxuxYPx9Z62g',
   'videoId': 'Tgeia3MrEHc',
   'parentID': '',
   'textDisplay': '앞으로 시에서 문제 생길때마다 각 시장들이 나와서 사과 유튜브 영상 만들면 진정성 느껴질듯',
   'textOriginal': '앞으로 시에서 문제 생길때마다 각 시장들이 나와서 사과 유튜브 영상 만들면 진정성 느껴질듯',
   'au

### Dart

url에 key와 value값을 추가하여 호출

site: https://opendart.fss.or.kr/ <br>
reference: https://opendart.fss.or.kr/guide/main.do?apiGrpCd=DS001 <br>

In [13]:
API_KEY = ''

#### 공시정보

```python
https://opendart.fss.or.kr/api/document.xml
```

In [42]:
tree = ET.parse('./CORPCODE.xml')
root = tree.getroot()

corp_code = pd.DataFrame()
for i, child in enumerate(tqdm(root)):
    data = {
        child[0].tag: child[0].text,
        child[1].tag: child[1].text,
        child[2].tag: child[2].text,
        child[3].tag: child[3].text,
    }

    corp_code = pd.concat([corp_code, pd.DataFrame(data, index=[i])])

100%|██████████| 107648/107648 [07:47<00:00, 230.35it/s]


#### 기업개황

```python
https://opendart.fss.or.kr/api/company.json?crtfc_key=xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx&corp_code=00126380
```

In [43]:
url = 'https://opendart.fss.or.kr/api/company.json?crtfc_key=03880acf9b9a1701f822dbbe25c4340eaaddfe57&corp_code=00126380'
response = requests.get(url)
json.loads(response.text)

{'status': '000',
 'message': '정상',
 'corp_code': '00126380',
 'corp_name': '삼성전자(주)',
 'corp_name_eng': 'SAMSUNG ELECTRONICS CO,.LTD',
 'stock_name': '삼성전자',
 'stock_code': '005930',
 'ceo_nm': '한종희',
 'corp_cls': 'Y',
 'jurir_no': '1301110006246',
 'bizr_no': '1248100998',
 'adres': '경기도 수원시 영통구  삼성로 129 (매탄동)',
 'hm_url': 'www.samsung.com/sec',
 'ir_url': '',
 'phn_no': '02-2255-0114',
 'fax_no': '031-200-7538',
 'induty_code': '264',
 'est_dt': '19690113',
 'acc_mt': '12'}

#### 공시검색

```
https://opendart.fss.or.kr/api/list.json?crtfc_key=xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx&bgn_de=20200117&end_de=20200117&corp_cls=Y&page_no=1&page_count=10
```

In [23]:
url = f'https://opendart.fss.or.kr/api/list.json?crtfc_key={API_KEY}&bgn_de=20240821&end_de=20240821&page_no=1&page_count=100'

In [24]:
response = requests.get(url)
results = json.loads(response.text)
results

{'status': '000',
 'message': '정상',
 'page_no': 1,
 'page_count': 100,
 'total_count': 229,
 'total_page': 3,
 'list': [{'corp_code': '01430475',
   'corp_name': '코람코라이프인프라리츠',
   'stock_code': '357120',
   'corp_cls': 'Y',
   'report_nm': '[기재정정]부동산투자회사부동산개발사업투자ㆍ운용              ',
   'rcept_no': '20240821800138',
   'flr_nm': '코람코라이프인프라리츠',
   'rcept_dt': '20240821',
   'rm': '유'},
  {'corp_code': '00163682',
   'corp_name': '메리츠증권',
   'stock_code': '008560',
   'corp_cls': 'E',
   'report_nm': '일괄신고추가서류(파생결합증권-주가연계증권)',
   'rcept_no': '20240821000182',
   'flr_nm': '메리츠증권',
   'rcept_dt': '20240821',
   'rm': ''},
  {'corp_code': '00136721',
   'corp_name': '신영증권',
   'stock_code': '001720',
   'corp_cls': 'Y',
   'report_nm': '투자설명서(일괄신고)',
   'rcept_no': '20240821000180',
   'flr_nm': '신영증권',
   'rcept_dt': '20240821',
   'rm': ''},
  {'corp_code': '00124504',
   'corp_name': '포스코인터내셔널',
   'stock_code': '047050',
   'corp_cls': 'Y',
   'report_nm': '자원개발진행사항공시(자율공시)              

In [22]:
pd.DataFrame(results.get('list'))

,corp_code,corp_name,stock_code,corp_cls,report_nm,rcept_no,flr_nm,rcept_dt,rm
0,00136721,신영증권,001720,Y,투자설명서(일괄신고),20240821000180,신영증권,20240821,
1,00124504,포스코인터내셔널,047050,Y,자원개발진행사항공시(자율공시),20240821800157,포스코인터내셔널,20240821,유
2,01430475,코람코라이프인프라리츠,357120,Y,[기재정정]증권신고서(집합투자증권-회사형)(주식회사코람코라이프인프라위탁관리부동산투자회사),20240821000020,코람코라이프인프라리츠,20240821,
3,01475609,디앤디플랫폼리츠,377190,Y,타법인주식및출자증권취득결정,20240821800130,디앤디플랫폼리츠,20240821,유
4,00120182,NH투자증권,005940,Y,투자설명서(일괄신고),20240821000172,NH투자증권,20240821,
5,00113359,교보증권,030610,Y,투자설명서(일괄신고),20240821000168,교보증권,20240821,
6,00111722,미래에셋증권,006800,Y,투자설명서(일괄신고),20240821000167,미래에셋증권,20240821,
7,00113359,교보증권,030610,Y,투자설명서(일괄신고),20240821000166,교보증권,20240821,
8,01475609,디앤디플랫폼리츠,377190,Y,주주명부폐쇄기간또는기준일설정,20240821800137,디앤디플랫폼리츠,20240821,유
9,00117601,유안타증권,003470,Y,일괄신고추가서류(기타파생결합사채),20240821000165,유안타증권,20240821,


In [27]:
url = f'https://opendart.fss.or.kr/api/list.json?crtfc_key={API_KEY}&bgn_de=20240821&end_de=20240821&page_no=1&page_count=100'
response = requests.get(url)
results = json.loads(response.text)
total_page = results.get('total_page')
information = results.get('list')

for page in range(2, total_page+1):
	url = f'https://opendart.fss.or.kr/api/list.json?crtfc_key={API_KEY}&bgn_de=20240821&end_de=20240821&page_no={page}&page_count=100'
	response = requests.get(url)
	results = json.loads(response.text)
	total_page = results.get('total_page')
	information.extend(results.get('list'))

In [28]:
len(information)

272

In [33]:
def extract_discloure(start_date, end_date):
	url = f'https://opendart.fss.or.kr/api/list.json?crtfc_key={API_KEY}&bgn_de={start_date}&end_de={end_date}&page_no=1&page_count=100'
	response = requests.get(url)
	results = json.loads(response.text)
	total_page = results.get('total_page')
	information = results.get('list')

	for page in tqdm(range(2, total_page+1)):
		url = f'https://opendart.fss.or.kr/api/list.json?crtfc_key={API_KEY}&bgn_de={start_date}&end_de={end_date}&page_no={page}&page_count=100'
		response = requests.get(url)
		results = json.loads(response.text)
		total_page = results.get('total_page')
		information.extend(results.get('list'))
	
	return information

In [35]:
results = extract_discloure('20240801', '20240821')

100%|██████████| 131/131 [01:03<00:00,  2.06it/s]


#### 재무재표

```python
https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json?crtfc_key=xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx&corp_code=00126380&bsns_year=2018&reprt_code=11011&fs_div=OFS
```

In [48]:
url = f'https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json?crtfc_key={API_KEY}&corp_code=00126380&bsns_year=2023&reprt_code=11011&fs_div=OFS'
response = requests.get(url)
result = json.loads(response.text)
pd.DataFrame(result.get('list'))

,rcept_no,reprt_code,bsns_year,corp_code,sj_div,sj_nm,account_id,account_nm,account_detail,thstrm_nm,thstrm_amount,frmtrm_nm,frmtrm_amount,bfefrmtrm_nm,bfefrmtrm_amount,ord,currency,thstrm_add_amount
0,20240312000736,11011,2023,00126380,BS,재무상태표,ifrs-full_Assets,자산총계,-,제 55 기,296857289000000,제 54 기,260083750000000,제 53 기,251112184000000,7,KRW,NaN
1,20240312000736,11011,2023,00126380,BS,재무상태표,ifrs-full_CurrentAssets,유동자산,-,제 55 기,68548442000000,제 54 기,59062658000000,제 53 기,73553416000000,8,KRW,NaN
2,20240312000736,11011,2023,00126380,BS,재무상태표,dart_ShortTermOtherReceivables,미수금,-,제 55 기,1910054000000,제 54 기,2925006000000,제 53 기,1832488000000,9,KRW,NaN
3,20240312000736,11011,2023,00126380,BS,재무상태표,dart_ShortTermPrepaidExpenses,선급비용,-,제 55 기,1349755000000,제 54 기,1047900000000,제 53 기,817689000000,10,KRW,NaN
4,20240312000736,11011,2023,00126380,BS,재무상태표,ifrs-full_CashAndCashEquivalents,현금및현금성자산,-,제 55 기,6061451000000,제 54 기,3921593000000,제 53 기,3918872000000,11,KRW,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110,20240312000736,11011,2023,00126380,SCE,자본변동표,ifrs-full_OtherComprehensiveIncomeNetOfTaxGain...,기타포괄손익-공정가치금융자산평가손익,자본 [구성요소]|이익잉여금 [구성요소],제 55 기,-12327000000,제 54 기,4340000000,제 53 기,,6,KRW,NaN
111,20240312000736,11011,2023,00126380,SCE,자본변동표,ifrs-full_OtherComprehensiveIncomeNetOfTaxGain...,순확정급여부채(자산) 재측정요소,자본 [구성요소]|기타자본구성요소 [구성요소],제 55 기,-572551000000,제 54 기,822001000000,제 53 기,-513309000000,7,KRW,NaN
112,20240312000736,11011,2023,00126380,SCE,자본변동표,ifrs-full_OtherComprehensiveIncomeNetOfTaxGain...,순확정급여부채(자산) 재측정요소,별도재무제표 [member],제 55 기,-572551000000,제 54 기,822001000000,제 53 기,-513309000000,7,KRW,NaN
113,20240312000736,11011,2023,00126380,SCE,자본변동표,ifrs-full_ProfitLoss,당기순이익(손실),별도재무제표 [member],제 55 기,25397099000000,제 54 기,25418778000000,제 53 기,30970954000000,8,KRW,NaN


In [47]:
url = f'https://opendart.fss.or.kr/api/fnlttSinglIndx.json?crtfc_key={API_KEY}&corp_code=00126380&bsns_year=2023&reprt_code=11011&idx_cl_code=M210000'
response = requests.get(url)
result = json.loads(response.text)
pd.DataFrame(result.get('list'))

,reprt_code,bsns_year,corp_code,stock_code,idx_cl_code,idx_cl_nm,idx_code,idx_nm,idx_val
0,11011,2023,00126380,005930,M210000,수익성지표,M211100,세전계속사업이익률,5.981
1,11011,2023,00126380,005930,M210000,수익성지표,M211200,순이익률,5.981
2,11011,2023,00126380,005930,M210000,수익성지표,M211250,총포괄이익률,7.275
3,11011,2023,00126380,005930,M210000,수익성지표,M211300,매출총이익률,30.335
4,11011,2023,00126380,005930,M210000,수익성지표,M211400,매출원가율,69.665
5,11011,2023,00126380,005930,M210000,수익성지표,M211550,ROE,4.311
6,11011,2023,00126380,005930,M210000,수익성지표,M211800,판관비율,27.798
7,11011,2023,00126380,005930,M210000,수익성지표,M212000,총자산영업이익률,1.452
8,11011,2023,00126380,005930,M210000,수익성지표,M212100,총자산세전계속사업이익률,3.425
9,11011,2023,00126380,005930,M210000,수익성지표,M212200,자기자본영업이익률,1.828


In [62]:
# 삼성전자, 하이닉스, 엔솔, 삼바, 현차, 셀트리온, 기아, KB금융, 신한지주
corp_codes = ['00126380', '00164779', '01515323', '00877059', '00164742', '00421045', '01664948', '00688996', '00382199']

indicators = []
financial_statements = []

for corp_code in tqdm(corp_codes):
    for bsns_year in range(2018, 2024):
        # 수익성지표
        url = f'https://opendart.fss.or.kr/api/fnlttSinglIndx.json?crtfc_key={API_KEY}&corp_code=00126380&bsns_year={bsns_year}&reprt_code=11011&idx_cl_code=M210000'
        response = requests.get(url)
        result = json.loads(response.text)
        if result.get('status') == '000':
            indicators.extend(result.get('list'))

        # 재무제표
        url = f'https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json?crtfc_key={API_KEY}&corp_code={corp_code}&bsns_year={bsns_year}&reprt_code=11011&fs_div=OFS'
        response = requests.get(url)
        result = json.loads(response.text)
        if result.get('status') == '000':
            financial_statements.extend(result.get('list'))

100%|██████████| 9/9 [00:12<00:00,  1.44s/it]


In [63]:
pd.DataFrame(indicators)

,reprt_code,bsns_year,corp_code,stock_code,idx_cl_code,idx_cl_nm,idx_code,idx_nm,idx_val
0,11011,2023,00126380,005930,M210000,수익성지표,M211100,세전계속사업이익률,5.981
1,11011,2023,00126380,005930,M210000,수익성지표,M211200,순이익률,5.981
2,11011,2023,00126380,005930,M210000,수익성지표,M211250,총포괄이익률,7.275
3,11011,2023,00126380,005930,M210000,수익성지표,M211300,매출총이익률,30.335
4,11011,2023,00126380,005930,M210000,수익성지표,M211400,매출원가율,69.665
...,...,...,...,...,...,...,...,...,...
130,11011,2023,00126380,005930,M210000,수익성지표,M212300,자기자본세전계속사업이익률,4.311
131,11011,2023,00126380,005930,M210000,수익성지표,M212400,자본금영업이익률,731.685
132,11011,2023,00126380,005930,M210000,수익성지표,M212500,자본금세전계속사업이익률,1725.555
133,11011,2023,00126380,005930,M210000,수익성지표,M212600,납입자본이익률,1725.555


In [66]:
pd.DataFrame(financial_statements)

,rcept_no,reprt_code,bsns_year,corp_code,sj_div,sj_nm,account_id,account_nm,account_detail,thstrm_nm,thstrm_amount,frmtrm_nm,frmtrm_amount,bfefrmtrm_nm,bfefrmtrm_amount,ord,currency,thstrm_add_amount
0,20190401004781,11011,2018,00126380,BS,재무상태표,ifrs_CurrentAssets,유동자산,-,제 50 기,80039455000000,제 49 기,70155189000000,제 48 기,69981128000000,1,KRW,NaN
1,20190401004781,11011,2018,00126380,BS,재무상태표,ifrs_CashAndCashEquivalents,현금및현금성자산,-,제 50 기,2607957000000,제 49 기,2763768000000,제 48 기,3778371000000,2,KRW,NaN
2,20190401004781,11011,2018,00126380,BS,재무상태표,dart_ShortTermDepositsNotClassifiedAsCashEquiv...,단기금융상품,-,제 50 기,34113871000000,제 49 기,25510064000000,제 48 기,30170656000000,3,KRW,NaN
3,20190401004781,11011,2018,00126380,BS,재무상태표,dart_ShortTermTradeReceivable,매출채권,-,제 50 기,24933267000000,제 49 기,27881777000000,제 48 기,23514012000000,4,KRW,NaN
4,20190401004781,11011,2018,00126380,BS,재무상태표,dart_ShortTermOtherReceivables,미수금,-,제 50 기,1515079000000,제 49 기,2201402000000,제 48 기,2319782000000,5,KRW,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3982,20240502000081,11011,2023,00382199,SCE,자본변동표,ifrs-full_Equity,자본총계,자본 [구성요소]|신종자본증권 [구성요소],제 23 기,4001731000000,제 22 기,4196968000000,제 21 기,3334531000000,25,KRW,NaN
3983,20240502000081,11011,2023,00382199,SCE,자본변동표,ifrs-full_Equity,자본총계,자본 [구성요소]|이익잉여금 [구성요소],제 23 기,7932131000000,제 22 기,8210445000000,제 21 기,8803435000000,25,KRW,NaN
3984,20240502000081,11011,2023,00382199,SCE,자본변동표,ifrs-full_Equity,자본총계,자본 [구성요소]|자본금 [구성요소],제 23 기,2969641000000,제 22 기,2969641000000,제 21 기,2969641000000,25,KRW,NaN
3985,20240502000081,11011,2023,00382199,SCE,자본변동표,ifrs-full_Equity,자본총계,자본 [구성요소]|자본잉여금 [구성요소],제 23 기,11350744000000,제 22 기,11350819000000,제 21 기,11350819000000,25,KRW,NaN


### 네이버

#### 네이버 트렌드

site: https://developers.naver.com/apps/#/list <br>
reference: [Naver Trend](https://developers.naver.com/docs/serviceapi/datalab/search/search.md#%ED%86%B5%ED%95%A9-%EA%B2%80%EC%83%89%EC%96%B4-%ED%8A%B8%EB%A0%8C%EB%93%9C)

<br>

<span style="font-size: 18px;">Naver Official Guideline</span>

```python
import os
import sys
import urllib.request
client_id = "YOUR_CLIENT_ID"
client_secret = "YOUR_CLIENT_SECRET"
url = "https://openapi.naver.com/v1/datalab/search";
body = "{\"startDate\":\"2017-01-01\",\"endDate\":\"2017-04-30\",\"timeUnit\":\"month\",\"keywordGroups\":[{\"groupName\":\"한글\",\"keywords\":[\"한글\",\"korean\"]},{\"groupName\":\"영어\",\"keywords\":[\"영어\",\"english\"]}],\"device\":\"pc\",\"ages\":[\"1\",\"2\"],\"gender\":\"f\"}";

request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id",client_id)
request.add_header("X-Naver-Client-Secret",client_secret)
request.add_header("Content-Type","application/json")
response = urllib.request.urlopen(request, data=body.encode("utf-8"))
rescode = response.getcode()
if(rescode==200):
    response_body = response.read()
    print(response_body.decode('utf-8'))
else:
    print("Error Code:" + rescode)
```

<br>

<span style="font-size: 18px;">requests.post</span>

```python
import requests
import json

url = "https://openapi.naver.com/v1/datalab/search"

headers = {
    'X-Naver-Client-Id': client_id,
    'X-Naver-Client-Secret': client_secret,
    'Content-Type': 'application/json',
}

body = {
    "startDate": "2017-01-01",
    "endDate": "2017-04-30",
    "timeUnit": "month",
    "keywordGroups":[
        {"groupName": "한글", "keywords": ["한글","korean"]},
        {"groupName": "영어", "keywords": ["영어","english"]}],
    "device": "pc",
    "ages": ["1", "2"],
    "gender": "f",
}

response = requests.post(url, data=json.dumps(body), headers=headers)
```

In [73]:
client_id = ''
client_secret = ''

In [80]:
import os
import sys
import urllib.request

client_id = client_id
client_secret = client_secret
url = "https://openapi.naver.com/v1/datalab/search"
body = "{\"startDate\":\"2017-01-01\",\"endDate\":\"2017-04-30\",\"timeUnit\":\"month\",\"keywordGroups\":[{\"groupName\":\"한글\",\"keywords\":[\"한글\",\"korean\"]},{\"groupName\":\"영어\",\"keywords\":[\"영어\",\"english\"]}],\"device\":\"pc\",\"ages\":[\"1\",\"2\"],\"gender\":\"f\"}";


request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id",client_id)
request.add_header("X-Naver-Client-Secret",client_secret)
request.add_header("Content-Type","application/json")
response = urllib.request.urlopen(request, data=body.encode("utf-8"))
rescode = response.getcode()
if(rescode==200):
    response_body = response.read()
    print(response_body.decode('utf-8'))
else:
    print("Error Code:" + rescode)

{"startDate":"2017-01-01","endDate":"2017-04-30","timeUnit":"month","results":[{"title":"한글","keywords":["한글","korean"],"data":[{"period":"2017-01-01","ratio":47.00101},{"period":"2017-02-01","ratio":53.23619},{"period":"2017-03-01","ratio":100},{"period":"2017-04-01","ratio":85.327}]},{"title":"영어","keywords":["영어","english"],"data":[{"period":"2017-01-01","ratio":40.0881},{"period":"2017-02-01","ratio":36.69942},{"period":"2017-03-01","ratio":52.11792},{"period":"2017-04-01","ratio":44.4595}]}]}


In [85]:
json.loads(body)

{'startDate': '2017-01-01',
 'endDate': '2017-04-30',
 'timeUnit': 'month',
 'keywordGroups': [{'groupName': '한글', 'keywords': ['한글', 'korean']},
  {'groupName': '영어', 'keywords': ['영어', 'english']}],
 'device': 'pc',
 'ages': ['1', '2'],
 'gender': 'f'}

In [81]:
json.loads(response_body.decode('utf-8'))

{'startDate': '2017-01-01',
 'endDate': '2017-04-30',
 'timeUnit': 'month',
 'results': [{'title': '한글',
   'keywords': ['한글', 'korean'],
   'data': [{'period': '2017-01-01', 'ratio': 47.00101},
    {'period': '2017-02-01', 'ratio': 53.23619},
    {'period': '2017-03-01', 'ratio': 100},
    {'period': '2017-04-01', 'ratio': 85.327}]},
  {'title': '영어',
   'keywords': ['영어', 'english'],
   'data': [{'period': '2017-01-01', 'ratio': 40.0881},
    {'period': '2017-02-01', 'ratio': 36.69942},
    {'period': '2017-03-01', 'ratio': 52.11792},
    {'period': '2017-04-01', 'ratio': 44.4595}]}]}

In [86]:
url = "https://openapi.naver.com/v1/datalab/search"

headers = {
    'X-Naver-Client-Id': client_id,
    'X-Naver-Client-Secret': client_secret,
    'Content-Type': 'application/json',
}

body = {
    "startDate": "2017-01-01",
    "endDate": "2017-04-30",
    "timeUnit": "month",
    "keywordGroups":[
        {"groupName": "한글", "keywords": ["한글","korean"]},
        {"groupName": "영어", "keywords": ["영어","english"]}],
    "device": "pc",
    "ages": ["1", "2"],
    "gender": "f",
}

response = requests.post(
    url, data=json.dumps(body), headers=headers)

In [89]:
json.loads(response.text)

{'startDate': '2017-01-01',
 'endDate': '2017-04-30',
 'timeUnit': 'month',
 'results': [{'title': '한글',
   'keywords': ['한글', 'korean'],
   'data': [{'period': '2017-01-01', 'ratio': 47.00101},
    {'period': '2017-02-01', 'ratio': 53.23619},
    {'period': '2017-03-01', 'ratio': 100},
    {'period': '2017-04-01', 'ratio': 85.327}]},
  {'title': '영어',
   'keywords': ['영어', 'english'],
   'data': [{'period': '2017-01-01', 'ratio': 40.0881},
    {'period': '2017-02-01', 'ratio': 36.69942},
    {'period': '2017-03-01', 'ratio': 52.11792},
    {'period': '2017-04-01', 'ratio': 44.4595}]}]}

## Data Sources

공공데이터포털: https://www.data.go.kr/ <br>
서울열린데이터광장: http://data.seoul.go.kr/ <br>
경제통계: https://ecos.bok.or.kr/ <br>
통계청: https://kosis.kr/index/index.do <br>
보건의료빅데이터개방시스템: https://opendata.hira.or.kr/home.do <br>
문화공공데이터광장: https://www.culture.go.kr/data/main/main.do <br>
VWORLD: https://www.vworld.kr/dev/v4api.do <br>
네이버 API: https://developers.naver.com/products/service-api/datalab/datalab.md <br>
카카오 API: https://developers.kakao.com/ <br>
대신증권 API: https://money2.daishin.com/E5/WTS/Customer/GuideTrading/DW_CybosPlus_Page.aspx?m=9508&p=8812&v=8632 <br>
미국공공데이터: https://www.data.gov/ <br>
